In [ ]:
#import libraries
import numpy as np
import pandas as pd
import os
import time

In [ ]:
# join the raw file folder
data_dir = os.path.join(os.getcwd(), "parquet_files")

In [ ]:
# load the file
def load_data(filename):
  return pd.read_parquet(filename)

# extract time only
# def extract_time(df):
#   df['intraday_time'] = df.index 
# create volume moving average
def create_volume_moving_average(df, candles = 8):
  df = df.copy()
  df[f"vma_{candles}"] = df["volume"].rolling(window=candles).mean()  
  return df

# resample logic
def resample_ohlcv(df, timeframe):
  return (df.resample(timeframe, label="right", closed="right").agg({
            "open": "first",
            "high": "max",
            "low": "min",
            "close": "last",
            "volume": "sum",
            "open_interest": "last"
        })
        .dropna(subset=["open"]))

# candle colour logic
def candlestick_colours(df):
  df = df.copy()
  df["candle"] = np.where(
    df["open"]>df["close"],"red", 
    np.where(df["open"]==df["close"],"white","green")
  )
  return df

# reversal logic
def reversal(df):
  df = df.copy()
  df["reversal"] = "None"
  bullish =( df["candle"].shift(1) == "red") & (df["candle"]=="green")
  bearish = (df["candle"].shift(1) == "green") & (df["candle"]=="red")
  df.loc[bullish, "reversal"] = "bullish"
  df.loc[bearish, "reversal"] = "bearish"
  return df

# englufing logic
def engulfing(df):
  df = df.copy()
  df["engulfing"] = "sluggish"
  bullish = (
      (df["reversal"] == "bullish") &
      (df["close"] > df["open"].shift(1)) &
      (df["open"] < df["close"].shift(1))
  )
  bearish = (
      (df["reversal"] == "bearish") &
      (df["close"] < df["open"].shift(1)) &
      (df["open"] > df["close"].shift(1))
  )
  df.loc[bullish, "engulfing"] = "bullish"
  df.loc[bearish, "engulfing"] = "bearish"
  return df

# volume logic
def volume_check(df, candles=8):
  df = df.copy()
  df["volume_check"] = "lower"
  df.loc[df["volume"] > df[f"vma_{candles}"], "volume_check"] = "higher"
  return df

# signal logic
def signal(df, ):
  df = df.copy()
  df['signal'] = 0
  long = (df['engulfing'].shift(1) == 'bullish') & (df['volume_check'].shift(1) == 'higher')
  short = (df['engulfing'].shift(1) == 'bearish') & (df['volume_check'].shift(1) == 'higher')
  df.loc[long, 'signal'] = 1
  df.loc[short, 'signal'] = -1
  return df

#entry logic
def entry(df):
  df = df.copy()
  df['entry_price'] = np.nan
  long = df['signal'] == 1
  short = df['signal'] == -1 
  df.loc[long | short, 'entry_price'] = df['open']
  return df

# sl logic
def stop_loss(df):
  df = df.copy()
  df['stop_loss'] = 0
  long = df['signal'] == 1
  short = df['signal'] == -1 
  df.loc[long, 'stop_loss'] = df['low'].shift(1)
  df.loc[short, 'stop_loss'] = df['high'].shift(1)
  return df

# risk in points
def risk_in_points(df):
  df = df.copy()
  df['risk_in_points']= np.nan
  long = df['signal'] == 1
  short = df['signal'] == -1 
  df.loc[long, 'risk_in_points'] = df['open'] - df['stop_loss']
  df.loc[short, 'risk_in_points'] =  df['stop_loss'] - df['open']
  return df 
#exit logic
def take_profit(df,rr= 2):
  df = df.copy()
  df['take_profit'] = np.nan
  long = df['signal'] == 1
  short = df['signal'] == -1 
  df.loc[long, 'take_profit'] = df['risk_in_points']*rr + df['open']
  df.loc[short, 'take_profit'] = df['open'] - df['risk_in_points']*rr
  return df

# quantity logic
def calc_quantity(df, monetary_risk = 1000):
  df = df.copy()
  df['quantity'] = 0
  signal_present = (df['signal'] == 1) | (df['signal'] == -1)
  df.loc[signal_present, 'quantity'] = np.floor(monetary_risk / df.loc[signal_present, 'risk_in_points'])
  return df

# trading simulation
def simulate_trade(df):
  df = df.copy()
  df['exit_reason'] = None
  df['exit_index'] = pd.NaT
  df['exit_price'] = np.nan
  i = 0
  for i in range(len(df)):
    if df.loc[df.index[i],'signal'] == 0:
      continue
    print(f"Processing i={i}/{len(df)}")
    entry = df.loc[df.index[i],'entry_price']
    sl = df.loc[df.index[i], 'stop_loss']
    tp = df.loc[df.index[i],'take_profit']
    direction = df.loc[df.index[i],'signal']
    
    # loop through each trade simulation
    for j in range(i+1, len(df)):
      high = df.iloc[j]['high']
      low = df.iloc[j]['low']
      signal = df.iloc[j]['signal']
      open_price = df.iloc[j]['open']
      if direction == 1:
        if signal == -1:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['reversal', df.index[j],open_price ]
          break
        if low<= sl:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['sl', df.index[j], sl]
          break  
        elif tp<= high:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['tp', df.index[j], tp]
          break
      if direction == -1:
        if signal == 1:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['reversal', df.index[j],open_price ]      
          break
        if high >= sl:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['sl', df.index[j], sl]
          break  
        elif tp>= low:
          df.loc[df.index[i],['exit_reason','exit_index','exit_price']]= ['tp', df.index[j], tp]
          break
  return df
    
  


  
  

In [ ]:
first_test = os.path.join(data_dir, "RELIANCE.parquet")
reliance = load_data(first_test)
reliance = resample_ohlcv(reliance, "5min")
reliance = create_volume_moving_average(reliance, 8)
reliance = candlestick_colours(reliance)
reliance = reversal(reliance) 
reliance = engulfing(reliance)
reliance = volume_check(reliance, candles = 8)
reliance = signal(reliance)
reliance = entry(reliance)
reliance = stop_loss(reliance)
reliance = risk_in_points(reliance)
reliance = take_profit(reliance, rr = 1)
reliance = calc_quantity(reliance, monetary_risk=1000)
reliance = simulate_trade(reliance)